<a href="https://colab.research.google.com/github/CakeNuthep/Create_Knowledge_Graph/blob/main/royin_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ทำการ Mount google drive เนื่องจากข้อมูล นักวิจัยและข้อมูล paper อยู่ใน Google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# Convert String to JSON

In [ ]:
path_file = '/content/drive/MyDrive/KnowledgeGraph/royin.json'

In [ ]:
# เนื่องจากข้อมูล royin.json ยังไม่ใช่รูปแบบ json ทีสมบูรณ์จึงต้องปรับแก้ข้อมูลเพื่อให้เป็นรูปแบบ json ที่สมบูรณ์
# ตัวอย่างข้อมูลที่มีเป็นประมาณดังนี้
# 
# {data}
# {data}
# {data}
#
# ทำการอ่านไฟล์ royin.json เพื่อทำการเพิ่ม [ ที่ด้านหน้าสุดของไฟล์ และเพิ่ม ] ที่ท้ายสุดของไฟล์ และแต่ละบรรทัดจะต้องคั่นด้วย ,
# ผลลัพธ์หลังจากปรับแก้ควรเป็น
#
#[
# {data},
# {data},
# {data}  
#]

f = open(path_file, "r",encoding="utf8")
count = 0
content = []
string_json = ''
begin = '['
end = ']'
split = ','

string_json += begin
while(True):
    # Read a line.
    line = f.readline()
    # When readline returns an empty string, the file is fully read.
    if line == "":
        print("::DONE::")
        break
    if count != 0:
      string_json += split
    string_json += line
    content.append(line)
    count +=1

string_json += end

::DONE::


In [ ]:
import json
#ทำการแปลงข้อมูล JSON เป็น Dictionary ของ Python
data = json.loads(string_json)

In [ ]:
data

[{'_id': '123',
  '_index': 'royin',
  '_score': 1,
  '_source': {'department': 'ธรรมศาสตร์และการเมือง',
   'name': 'ไพฑูรย์ พงศะบุตร (ราชบัณฑิต)',
   'subject': 'สังคมศาสตร์',
   'title': 'ศาสตราจารย์กิตติคุณ'},
  '_type': 'royin'},
 {'_id': '120',
  '_index': 'royin',
  '_score': 1,
  '_source': {'department': 'วิทยาศาสตร์',
   'name': 'ไพรัช ธัชยพงษ์ (ภาคีสมาชิก)',
   'subject': 'วิทยาศาสตร์ประยุกต์',
   'title': 'ศาสตราจารย์ ดร.'},
  '_type': 'royin'},
 {'_id': '73',
  '_index': 'royin',
  '_score': 1,
  '_source': {'department': 'วิทยาศาสตร์',
   'name': 'วิสุทธิ์ ใบไม้ (ราชบัณฑิต)',
   'subject': 'วิทยาศาสตร์ชีวภาพ',
   'title': 'ศาสตราจารย์เกียรติคุณ ดร.'},
  '_type': 'royin'},
 {'_id': '92',
  '_index': 'royin',
  '_score': 1,
  '_source': {'department': 'วิทยาศาสตร์',
   'name': 'ระวี ภาวิไล (ราชบัณฑิต)',
   'subject': 'วิทยาศาสตร์กายภาพ',
   'title': 'ศาสตราจารย์กิตติคุณ ดร.'},
  '_type': 'royin'},
 {'_id': '60',
  '_index': 'royin',
  '_score': 1,
  '_source': {'department':

In [ ]:
# ทำการแปลง Dictionary เป็น Dataframe
df_publish = pd.json_normalize(data)

In [ ]:
df_publish

,_index,_type,_id,_score,_source.subject,_source.name,_source.title,_source.department
0,royin,royin,123,1,สังคมศาสตร์,ไพฑูรย์ พงศะบุตร (ราชบัณฑิต),ศาสตราจารย์กิตติคุณ,ธรรมศาสตร์และการเมือง
1,royin,royin,120,1,วิทยาศาสตร์ประยุกต์,ไพรัช ธัชยพงษ์ (ภาคีสมาชิก),ศาสตราจารย์ ดร.,วิทยาศาสตร์
2,royin,royin,73,1,วิทยาศาสตร์ชีวภาพ,วิสุทธิ์ ใบไม้ (ราชบัณฑิต),ศาสตราจารย์เกียรติคุณ ดร.,วิทยาศาสตร์
3,royin,royin,92,1,วิทยาศาสตร์กายภาพ,ระวี ภาวิไล (ราชบัณฑิต),ศาสตราจารย์กิตติคุณ ดร.,วิทยาศาสตร์
4,royin,royin,60,1,สังคมศาสตร์,สนั่น ปัทมะทิน,ศาสตราจารย์,ธรรมศาสตร์และการเมือง
...,...,...,...,...,...,...,...,...
214,royin,royin,53,1,ปรัชญา,สิทธิ์ บุตรอินทร์ (ภาคีสมาชิก),ศาสตราจารย์ ดร.,ธรรมศาสตร์และการเมือง
215,royin,royin,68,1,วิจิตรศิลป์,วิรุณ ตั้งเจริญ (ภาคีสมาชิก),ศาสตราจารย์ ดร.,ศิลปกรรม
216,royin,royin,17,1,ปรัชญา,วรรณไวทยากร วรวรรณ,พลตรี พระเจ้าวรวงศ์เธอ กรมหมื่นนราธิปพงศ์ประพั...,ธรรมศาสตร์และการเมือง
217,royin,royin,31,1,วิทยาศาสตร์ชีวภาพ,สิริวัฒน์ วงษ์ศิริ (ภาคีสมาชิก),ศาสตราจารย์ ดร.,วิทยาศาสตร์


In [ ]:
df_new = pd.concat([df_publish])

# clean name

In [ ]:
def front(name, title):
    if name.find(title)>-1 and name.startswith(title):
        a = []
        b = ''
        for i in name:
            a.append(i)
        for i in range(len(title)):
            del a[0]
        for i in a:
            b = b+i
    return b 

def behind(name, title):
    if name.find(title)>-1 and name.endswith(title):
        a = []
        b = ''
        for i in name:
            a.append(i)
        for i in range(name.rfind(title),len(name)):
            del a[name.rfind(title)]
        for i in a:
            b = b+i
    return b

In [ ]:
def deltitlefull(str):
    a = str
    b = []
    b.append(a)
    prefix = ''
    for i in b:
        if i in ('ว่าที่','จอมพล','พลเอก','พลโท','พลตรี','พลจัตวา','พันเอก','พันโท','พันตรี','ร้อยเอก','ร้อยโท','ร้อยตรี','จ่าสิบเอกพิเศษ','จ่าสิบเอก','จ่าสิบโท','จ่าสิบตรี','สิบเอก','สิบโท','สิบตรี','จอมพลเรือ','พลเรือเอก','พลเรือโท','พลเรือตรี','พลเรือจัตวา','นาวาเอก','นาวาโท','นาวาตรี','เรือเอก','เรือโท','เรือตรี','พันจ่าเอก','พันจ่าโท','พันจ่าตรี','จ่าเอก','จ่าโท','จ่าตรี','จอมพลอากาศ','พลอากาศเอก','พลอากาศโท','พลอากาศตรี','พลอากาศจัตวา','นาวาอากาศเอก','นาวาอากาศโท','นาวาอากาศตรี','เรืออากาศเอก','เรืออากาศโท','เรืออากาศตรี','พันจ่าอากาศเอก','พันจ่าอากาศโท','พันจ่าอากาศตรี','จ่าอากาศเอก','จ่าอากาศโท','จ่าอากาศตรี','พลตำรวจเอก','พลตำรวจโท','พลตำรวจตรี','พลตำรวจจัตวา','พันตำรวจเอก','พันตำรวจโท','พันตำรวจตรี','ร้อยตำรวจเอก','ร้อยตำรวจโท','ร้อยตำรวจตรี','ดาบตำรวจ','จ่าสิบตำรวจ','สิบตำรวจเอก','สิบตำรวจโท','สิบตำรวจตรี','หม่อมหลวง','ผู้ช่วยศาสตราจารย์','รองศาสตราจารย์','ศาสตราจารย์','อาจารย์','แพทย์หญิง','นายแพทย์','นาง','นางสาว','นาย'):
            return a, prefix
    c = ['ฯพณฯ']
    c1 = ['ผู้ช่วยศาสตราจารย์พิเศษ','รองศาสตราจารย์พิเศษ','ศาสตราจารย์พิเศษ','อาจารย์พิเศษ','ผู้ช่วยศาสตราจารย์','รองศาสตราจารย์','ศาสตราจารย์กิตติคุณ','ศาสตราจารย์เกียรติยศ','ศาสตราจารย์','อาจารย์']
    c2 = ['ว่าที่']
    c3 = ['จอมพลพิเศษหญิง','พลเอกพิเศษหญิง','พลโทพิเศษหญิง','พลตรีพิเศษหญิง','พลจัตวาพิเศษหญิง','พันเอกพิเศษหญิง','พันโทพิเศษหญิง','พันตรีพิเศษหญิง','ร้อยเอกพิเศษหญิง','ร้อยโทพิเศษหญิง','ร้อยตรีพิเศษหญิง','จ่าสิบเอกพิเศษพิเศษหญิง','จ่าสิบเอกพิเศษหญิง','จ่าสิบโทพิเศษหญิง','จ่าสิบตรีพิเศษหญิง','สิบเอกพิเศษหญิง','สิบโทพิเศษหญิง','สิบตรีพิเศษหญิง','จอมพลเรือพิเศษหญิง','พลเรือเอกพิเศษหญิง','พลเรือโทพิเศษหญิง','พลเรือตรีพิเศษหญิง','พลเรือจัตวาพิเศษหญิง','นาวาเอกพิเศษหญิง','นาวาโทพิเศษหญิง','นาวาตรีพิเศษหญิง','เรือเอกพิเศษหญิง','เรือโทพิเศษหญิง','เรือตรีพิเศษหญิง','พันจ่าเอกพิเศษหญิง','พันจ่าโทพิเศษหญิง','พันจ่าตรีพิเศษหญิง','จ่าเอกพิเศษหญิง','จ่าโทพิเศษหญิง','จ่าตรีพิเศษหญิง','จอมพลอากาศพิเศษหญิง','พลอากาศเอกพิเศษหญิง','พลอากาศโทพิเศษหญิง','พลอากาศตรีพิเศษหญิง','พลอากาศจัตวาพิเศษหญิง','นาวาอากาศเอกพิเศษหญิง','นาวาอากาศโทพิเศษหญิง','นาวาอากาศตรีพิเศษหญิง','เรืออากาศเอกพิเศษหญิง','เรืออากาศโทพิเศษหญิง','เรืออากาศตรีพิเศษหญิง','พันจ่าอากาศเอกพิเศษหญิง','พันจ่าอากาศโทพิเศษหญิง','พันจ่าอากาศตรีพิเศษหญิง','จ่าอากาศเอกพิเศษหญิง','จ่าอากาศโทพิเศษหญิง','จ่าอากาศตรีพิเศษหญิง','พลตำรวจเอกพิเศษหญิง','พลตำรวจโทพิเศษหญิง','พลตำรวจตรีพิเศษหญิง','พลตำรวจจัตวาพิเศษหญิง','พันตำรวจเอกพิเศษหญิง','พันตำรวจโทพิเศษหญิง','พันตำรวจตรีพิเศษหญิง','ร้อยตำรวจเอกพิเศษหญิง','ร้อยตำรวจโทพิเศษหญิง','ร้อยตำรวจตรีพิเศษหญิง','ดาบตำรวจพิเศษหญิง','จ่าสิบตำรวจพิเศษหญิง','สิบตำรวจเอกพิเศษหญิง','สิบตำรวจโทพิเศษหญิง','สิบตำรวจตรีพิเศษหญิง','จอมพลพิเศษ','พลเอกพิเศษ','พลโทพิเศษ','พลตรีพิเศษ','พลจัตวาพิเศษ','พันเอกพิเศษ','พันโทพิเศษ','พันตรีพิเศษ','ร้อยเอกพิเศษ','ร้อยโทพิเศษ','ร้อยตรีพิเศษ','จ่าสิบเอกพิเศษพิเศษ','จ่าสิบเอกพิเศษ','จ่าสิบโทพิเศษ','จ่าสิบตรีพิเศษ','สิบเอกพิเศษ','สิบโทพิเศษ','สิบตรีพิเศษ','จอมพลเรือพิเศษ','พลเรือเอกพิเศษ','พลเรือโทพิเศษ','พลเรือตรีพิเศษ','พลเรือจัตวาพิเศษ','นาวาเอกพิเศษ','นาวาโทพิเศษ','นาวาตรีพิเศษ','เรือเอกพิเศษ','เรือโทพิเศษ','เรือตรีพิเศษ','พันจ่าเอกพิเศษ','พันจ่าโทพิเศษ','พันจ่าตรีพิเศษ','จ่าเอกพิเศษ','จ่าโทพิเศษ','จ่าตรีพิเศษ','จอมพลอากาศพิเศษ','พลอากาศเอกพิเศษ','พลอากาศโทพิเศษ','พลอากาศตรีพิเศษ','พลอากาศจัตวาพิเศษ','นาวาอากาศเอกพิเศษ','นาวาอากาศโทพิเศษ','นาวาอากาศตรีพิเศษ','เรืออากาศเอกพิเศษ','เรืออากาศโทพิเศษ','เรืออากาศตรีพิเศษ','พันจ่าอากาศเอกพิเศษ','พันจ่าอากาศโทพิเศษ','พันจ่าอากาศตรีพิเศษ','จ่าอากาศเอกพิเศษ','จ่าอากาศโทพิเศษ','จ่าอากาศตรีพิเศษ','พลตำรวจเอกพิเศษ','พลตำรวจโทพิเศษ','พลตำรวจตรีพิเศษ','พลตำรวจจัตวาพิเศษ','พันตำรวจเอกพิเศษ','พันตำรวจโทพิเศษ','พันตำรวจตรีพิเศษ','ร้อยตำรวจเอกพิเศษ','ร้อยตำรวจโทพิเศษ','ร้อยตำรวจตรีพิเศษ','ดาบตำรวจพิเศษ','จ่าสิบตำรวจพิเศษ','สิบตำรวจเอกพิเศษ','สิบตำรวจโทพิเศษ','สิบตำรวจตรีพิเศษ','จอมพลหญิง','พลเอกหญิง','พลโทหญิง','พลตรีหญิง','พลจัตวาหญิง','พันเอกหญิง','พันโทหญิง','พันตรีหญิง','ร้อยเอกหญิง','ร้อยโทหญิง','ร้อยตรีหญิง','จ่าสิบเอกพิเศษหญิง','จ่าสิบเอกหญิง','จ่าสิบโทหญิง','จ่าสิบตรีหญิง','สิบเอกหญิง','สิบโทหญิง','สิบตรีหญิง','จอมพลเรือหญิง','พลเรือเอกหญิง','พลเรือโทหญิง','พลเรือตรีหญิง','พลเรือจัตวาหญิง','นาวาเอกหญิง','นาวาโทหญิง','นาวาตรีหญิง','เรือเอกหญิง','เรือโทหญิง','เรือตรีหญิง','พันจ่าเอกหญิง','พันจ่าโทหญิง','พันจ่าตรีหญิง','จ่าเอกหญิง','จ่าโทหญิง','จ่าตรีหญิง','จอมพลอากาศหญิง','พลอากาศเอกหญิง','พลอากาศโทหญิง','พลอากาศตรีหญิง','พลอากาศจัตวาหญิง','นาวาอากาศเอกหญิง','นาวาอากาศโทหญิง','นาวาอากาศตรีหญิง','เรืออากาศเอกหญิง','เรืออากาศโทหญิง','เรืออากาศตรีหญิง','พันจ่าอากาศเอกหญิง','พันจ่าอากาศโทหญิง','พันจ่าอากาศตรีหญิง','จ่าอากาศเอกหญิง','จ่าอากาศโทหญิง','จ่าอากาศตรีหญิง','พลตำรวจเอกหญิง','พลตำรวจโทหญิง','พลตำรวจตรีหญิง','พลตำรวจจัตวาหญิง','พันตำรวจเอกหญิง','พันตำรวจโทหญิง','พันตำรวจตรีหญิง','ร้อยตำรวจเอกหญิง','ร้อยตำรวจโทหญิง','ร้อยตำรวจตรีหญิง','ดาบตำรวจหญิง','จ่าสิบตำรวจหญิง','สิบตำรวจเอกหญิง','สิบตำรวจโทหญิง','สิบตำรวจตรีหญิง','จอมพล','พลเอก','พลโท','พลตรี','พลจัตวา','พันเอก','พันโท','พันตรี','ร้อยเอก','ร้อยโท','ร้อยตรี','จ่าสิบเอกพิเศษ','จ่าสิบเอก','จ่าสิบโท','จ่าสิบตรี','สิบเอก','สิบโท','สิบตรี','จอมพลเรือ','พลเรือเอก','พลเรือโท','พลเรือตรี','พลเรือจัตวา','นาวาเอก','นาวาโท','นาวาตรี','เรือเอก','เรือโท','เรือตรี','พันจ่าเอก','พันจ่าโท','พันจ่าตรี','จ่าเอก','จ่าโท','จ่าตรี','จอมพลอากาศ','พลอากาศเอก','พลอากาศโท','พลอากาศตรี','พลอากาศจัตวา','นาวาอากาศเอก','นาวาอากาศโท','นาวาอากาศตรี','เรืออากาศเอก','เรืออากาศโท','เรืออากาศตรี','พันจ่าอากาศเอก','พันจ่าอากาศโท','พันจ่าอากาศตรี','จ่าอากาศเอก','จ่าอากาศโท','จ่าอากาศตรี','พลตำรวจเอก','พลตำรวจโท','พลตำรวจตรี','พลตำรวจจัตวา','พันตำรวจเอก','พันตำรวจโท','พันตำรวจตรี','ร้อยตำรวจเอก','ร้อยตำรวจโท','ร้อยตำรวจตรี','ดาบตำรวจ','จ่าสิบตำรวจ','สิบตำรวจเอก','สิบตำรวจโท','สิบตำรวจตรี']   
    c4 = ['แพทย์หญิง','นายแพทย์','ทันตแพทย์หญิง','ทันตแพทย์','นางสาว','นาง','นาย']
    c5 = ['หม่อมหลวง','คุณหญิง','ท่านหญิง','ท่านผู้หญิง','หม่อมราชวงศ์หญิง','หม่อมราชวงศ์','หม่อมเจ้าหญิง','หม่อมเจ้า']


    for i in c:
        if a.startswith(i) == True:
            a = front(a,i)
            prefix = i
            break
    for i in c1:
        if a.startswith(i) == True:
            a = front(a,i)
            prefix = i
            break
    for i in c2:
        if a.startswith(i) == True:
            a = front(a,i)
            prefix = i
            break
    for i in c3:
        if a.startswith(i) == True:
            a = front(a,i)
            prefix = i
            break
    for i in c4:
        if a.startswith(i) == True:
            a = front(a,i)
            prefix = i
            break
    for i in c5:
        if a.startswith(i) == True:
            a = front(a,i)
            prefix = i
            break
    return a, prefix
   
def deltitleshort(str):
    a = str
    b = ''
    s  = ['พล.อ.','พล.ท.','พล.ต','พล.จ.','พ.อ.','พ.ท.','พ.ต.','ร.อ.','ร.ท.','ร.ต.','จ.ส.อ.','จ.ส.ท.','จ.ส.ต.','ส.อ.','ส.ท.','ส.ต.','พล.ร.อ.','พล.ร.ท.','พล.ร.ต.','พล.ร.จ.','น.อ.','น.ท.','น.ต.','พ.จ.อ.','พ.จ.ท.','พ.จ.ต.','จ.อ.','จ.ท.','จ.ต.','พล.อ.อ.','พล.อ.ท.','พล.อ.ต.','พล.อ.จ.','พ.อ.อ.','พ.อ.ท.','พ.อ.ต.','พล.ต.อ.','พล.ต.ท.','พล.ต.ต.','พล.ต.จ.','พ.ต.อ.','พ.ต.ท.','พ.ต.ต.','ร.ต.อ.','ร.ต.ท.','ร.ต.ต.','ด.ต.','จ.ส.ต.','ส.ต.อ.','ส.ต.ท.','ส.ต.ต.','พญ.','นพ.','ดร.','น.ส.','ม.ล.','ม.ร.ว','ม.จ.','ผศ.','รศ.','ศ.','อ.']
    s1 = ('พญ.','นพ.','ดร.','น.ส.','น.สพ.','สพ.ญ.','ภญ.','ภกญ.')
    s2 = ('ผศ.','รศ.','ศ.','อ.')
    s3 = ('ม.ล.','ม.ร.ว','ม.จ.')
    for i in range(0,a.rfind('.')+1):
        b = b+a[i]

    prefix = ''
    for i in s:
        if b.endswith(i) == True:
            a = front(a,b)
            prefix = b
            for i in s1:
                if b.endswith(i) == True:
                    return a, prefix
            for i in s2:
                if b.endswith(i) == True:
                    if a.startswith('(พิเศษ)') == True:
                        a = front(a,'(พิเศษ)')
                        prefix += '(พิเศษ)'
                        return a, prefix
                    elif a.startswith('พิเศษ') == True:
                        a = front(a,'พิเศษ')
                        prefix += 'พิเศษ'
                        return a, prefix
                    else:
                        return a, prefix
            for i in s3:
                if b.endswith(i) == True:
                    if a.startswith('หญิง') == True:
                        a = front(a,'หญิง')
                        prefix += ('หญิง')
                        return a, prefix
                    elif a.startswith('(หญิง)') == True:
                        a = front(a,'(หญิง)')
                        prefix += '(หญิง)'
                        return a, prefix
            if a.startswith('(พิเศษ)') == True:
                a = front(a,'(พิเศษ)')
                prefix += '(พิเศษ)'
            elif a.startswith('พิเศษ') == True:
                a = front(a,'พิเศษ')
                prefix += 'พิเศษ'
            if a.startswith('หญิง') == True:
                a = front(a,'หญิง')
                prefix += 'หญิง'
            elif a.startswith('(หญิง)') == True:
                a = front(a,'(หญิง)')
                prefix += '(หญิง)'
            return a, prefix
    else:
        return a, prefix

def delnumber(str):
    p = ['0','1','2','3','4','5','6','7','8','9']
    a = []
    b = ''
    j = 0
    for i in str:
        a.append(i)
    while j < len(a):
        if a[j] in p:
            del a[j]
            j = j-1
        j = j+1   
    for i in a:
        b = b + i
    return b

def celanSpace(str):
    str = " ".join(str.split())
    return str.strip()

def delengtitle(str):
    a = str
    b= ''
    # c = ('Md.','MD.','md.','Mrs.','MRS.','mrs.','Ph.D.','Associate.','Assoc.','Asso.','Asst.','Assist.','Professor.','Prof.','Dr.','Mr.','Ms.','associate.','assoc.','asso.','asst.','assist.','professor.','prof.','dr.','mr.','ms.','ASSOCIATE.','ASSOC.','ASSO.','ASST.','ASSIST.','PROFESSOR.','PROF.','DR.','MR.','MS.',
    #      'Md ','MD ','md ','Mrs ','MRS ','mrs ','Ph.D ','Associate ','Assoc ','Asso ','Asst ','Assist ','Professor ','Prof ','Dr ','Mr ','Ms ','associate ','assoc ','asso ','asst ','assist ','professor ','prof ','dr ','mr ','ms ','ASSOCIATE ','ASSOC ','ASSO ','ASST ','ASSIST ','PROFESSOR ','PROF ','DR ','MR ','MS ')
    c = ('Associate.','Assoc.','Asso.','Asst.','Assist.','associate.','assoc.','asso.','asst.','assist.','ASSOCIATE.','ASSOC.','ASSO.','ASST.','ASSIST.','Associate ','Assoc ','Asso ','Asst ','Assist ','associate ','assoc ','asso ','asst ','assist ','ASSOCIATE ','ASSOC ','ASSO ','ASST ','ASSIST ',
          'Ph.D.','Professor.','Prof.','professor.','prof.','dr.','PROFESSOR.','PROF.','Ph.D ','Professor ','Prof ','professor ','prof ','PROFESSOR ','PROF ',
          'Dr.','DR.','Dr ','dr ','DR ',
          'Md.','MD.','md.','Md ','MD ','md ',
          'Mrs.','MRS.','mrs.','Mr.','Ms.','mr.','ms.','MR.','MS.','Mrs ','MRS ','mrs ','Mr ','Ms ','mr ','ms ','MR ','MS ')
    for i in range(0,a.rfind('.')+1):
        b += a[i]

    prefix = ''
    
    found = True
    while found:
      found = False
      for i in c:
          if a.find(i) != -1:
              # a = front(a,b)
              a = a.replace(i,"")
              found = True
              prefix = prefix + i
              break
    return a, prefix

In [ ]:
def delAllPrefix(str):
  str = delnumber(str)
  str, prefix1 = deltitlefull(str)
  str, prefix2 = deltitleshort(str)
  str, prefix3 = delengtitle(str)
  str = celanSpace(str)
  prefix = prefix1+prefix2+prefix3
  return str, prefix

def cleanName(row):
  clean_name, prefix = delAllPrefix(row['_source.name'])
  clean_name = clean_name.replace(',','')
  row['clean_name'] = clean_name
  row['prefix'] = prefix
  return row

In [ ]:
#ทำการจัดการคำ prefix ที่นำหน้าชื่อแล้วเอาค่าที่ถูกกำจัดแล้วมาใส่ในคอลัมน์ใหม่ที่ชื่อว่า clean_name
#เช่น หากชื่อเป็น 
# ดร. บังอร ศุภวิทิตพัฒนา จะกลายเป็น บังอร ศุภวิทิตพัฒนา 
df_new = df_new.apply(cleanName,axis=1)

In [ ]:
df_new

,_index,_type,_id,_score,_source.subject,_source.name,_source.title,_source.department,clean_name,prefix
0,royin,royin,123,1,สังคมศาสตร์,ไพฑูรย์ พงศะบุตร (ราชบัณฑิต),ศาสตราจารย์กิตติคุณ,ธรรมศาสตร์และการเมือง,ไพฑูรย์ พงศะบุตร (ราชบัณฑิต),
1,royin,royin,120,1,วิทยาศาสตร์ประยุกต์,ไพรัช ธัชยพงษ์ (ภาคีสมาชิก),ศาสตราจารย์ ดร.,วิทยาศาสตร์,ไพรัช ธัชยพงษ์ (ภาคีสมาชิก),
2,royin,royin,73,1,วิทยาศาสตร์ชีวภาพ,วิสุทธิ์ ใบไม้ (ราชบัณฑิต),ศาสตราจารย์เกียรติคุณ ดร.,วิทยาศาสตร์,วิสุทธิ์ ใบไม้ (ราชบัณฑิต),
3,royin,royin,92,1,วิทยาศาสตร์กายภาพ,ระวี ภาวิไล (ราชบัณฑิต),ศาสตราจารย์กิตติคุณ ดร.,วิทยาศาสตร์,ระวี ภาวิไล (ราชบัณฑิต),
4,royin,royin,60,1,สังคมศาสตร์,สนั่น ปัทมะทิน,ศาสตราจารย์,ธรรมศาสตร์และการเมือง,สนั่น ปัทมะทิน,
...,...,...,...,...,...,...,...,...,...,...
214,royin,royin,53,1,ปรัชญา,สิทธิ์ บุตรอินทร์ (ภาคีสมาชิก),ศาสตราจารย์ ดร.,ธรรมศาสตร์และการเมือง,สิทธิ์ บุตรอินทร์ (ภาคีสมาชิก),
215,royin,royin,68,1,วิจิตรศิลป์,วิรุณ ตั้งเจริญ (ภาคีสมาชิก),ศาสตราจารย์ ดร.,ศิลปกรรม,วิรุณ ตั้งเจริญ (ภาคีสมาชิก),
216,royin,royin,17,1,ปรัชญา,วรรณไวทยากร วรวรรณ,พลตรี พระเจ้าวรวงศ์เธอ กรมหมื่นนราธิปพงศ์ประพั...,ธรรมศาสตร์และการเมือง,วรรณไวทยากร วรวรรณ,
217,royin,royin,31,1,วิทยาศาสตร์ชีวภาพ,สิริวัฒน์ วงษ์ศิริ (ภาคีสมาชิก),ศาสตราจารย์ ดร.,วิทยาศาสตร์,สิริวัฒน์ วงษ์ศิริ (ภาคีสมาชิก),


# Create Author Table

In [ ]:
# ทำการดึงชื่อคนจากคอลัมน์ที่ชื่อว่า clean_name ที่ไม่ซ้ำกันออกมาใส่ใน array ที่ชือว่า uniq_name
uniq_name = df_new['clean_name'].unique()

In [ ]:
uniq_name

array(['ไพฑูรย์ พงศะบุตร (ราชบัณฑิต)', 'ไพรัช ธัชยพงษ์ (ภาคีสมาชิก)',
       'วิสุทธิ์ ใบไม้ (ราชบัณฑิต)', 'ระวี ภาวิไล (ราชบัณฑิต)',
       'สนั่น ปัทมะทิน', 'สมชาติ โสภณรณฤทธิ์ (ราชบัณฑิต)',
       'โสภา ชูพิกุลชัย ชปีลมันน์ (ราชบัณฑิต)',
       'สิวลี ศิริไล (ภาคีสมาชิก)', 'สมใจ นิ่มเล็ก',
       'มนุวดี หังสพฤกษ์ (ราชบัณฑิต)', 'พรสรรค์ วัฒนางกูร (ภาคีสมาชิก)',
       'ยงค์วิมล เลณบุรี (ราชบัณฑิต)', 'เสฐียร พันธรังษี',
       'สวัสดิ์ ตันติสุข', 'ประเวศ วะสี (ราชบัณฑิตกิตติมศักดิ์)',
       'วรศักดิ์ กนกนุกูลชัย (ภาคีสมาชิก)', 'ประเสริฐ ณ นคร (ราชบัณฑิต)',
       'พูนพิศ อมาตยกุล (ราชบัณฑิตกิตติมศักดิ์)',
       'พูนทรัพย์ นพวงศ์ ณ อยุธยา (ราชบัณฑิตกิตติมศักดิ์)',
       'ไขแสง ศุขะวัฒนะ (ราชบัณฑิต)', 'ปกรณ์ อดุลพันธุ์ (ราชบัณฑิต)',
       'ผลิน ภู่จรูญ (ราชบัณฑิต)', 'สมชัย บวรกิตติ (ราชบัณฑิต)',
       'ปณต มิคะเสน (ราชบัณฑิต)', 'ลิขิต ธีรเวคิน (ราชบัณฑิต)',
       'ยง ภู่วรวรรณ (ภาคีสมาชิก)', 'วิชิตวงศ์ ณ ป้อมเพชร (ราชบัณฑิต)',
       'กุหลาบ มัลลิกะมาส', 'มีชัย ฤชุพันธุ์ (ราชบัณฑ

In [ ]:
#ทำการแปลง array เป็น Dataframe
df_unique_name = pd.DataFrame(uniq_name,columns=['clean_name'])

In [ ]:
df_unique_name

,clean_name
0,ไพฑูรย์ พงศะบุตร (ราชบัณฑิต)
1,ไพรัช ธัชยพงษ์ (ภาคีสมาชิก)
2,วิสุทธิ์ ใบไม้ (ราชบัณฑิต)
3,ระวี ภาวิไล (ราชบัณฑิต)
4,สนั่น ปัทมะทิน
...,...
214,สิทธิ์ บุตรอินทร์ (ภาคีสมาชิก)
215,วิรุณ ตั้งเจริญ (ภาคีสมาชิก)
216,วรรณไวทยากร วรวรรณ
217,สิริวัฒน์ วงษ์ศิริ (ภาคีสมาชิก)


In [ ]:
# นำค่า index มาใส่ในคอลัมน์ใหม่ที่ชื่อว่า id โดยให้ค่าเริ่มต้นที่ 542598
start_id = 542598
df_unique_name['id'] = df_unique_name.index + start_id

In [ ]:
df_unique_name

,clean_name,id
0,ไพฑูรย์ พงศะบุตร (ราชบัณฑิต),542598
1,ไพรัช ธัชยพงษ์ (ภาคีสมาชิก),542599
2,วิสุทธิ์ ใบไม้ (ราชบัณฑิต),542600
3,ระวี ภาวิไล (ราชบัณฑิต),542601
4,สนั่น ปัทมะทิน,542602
...,...,...
214,สิทธิ์ บุตรอินทร์ (ภาคีสมาชิก),542812
215,วิรุณ ตั้งเจริญ (ภาคีสมาชิก),542813
216,วรรณไวทยากร วรวรรณ,542814
217,สิริวัฒน์ วงษ์ศิริ (ภาคีสมาชิก),542815


# df_author

In [ ]:
def is_english(str):
    for i in str:
        if i in ('A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z'):
            return True
    else:    
        return False

def is_thai(str):
    for i in str:
        if i in ('ก','ข','ฃ','ค','ฅ','ฆ','ง','จ','ฉ','ช','ซ','ฌ','ญ','ฎ','ฏ','ฐ','ฑ','ฒ','ณ','ด','ต','ถ','ท','ธ','น','บ','ป','ผ','ฝ','พ','ฟ','ภ','ม','ย','ร','ล','ว','ศ','ษ','ส','ห','ฬ','อ','ฮ'):
            return True
    else:
        return False

def language(text):
    if is_english(text):
      return 'eng'
    elif is_thai(text):
      return 'th'
    else:
      return 'other'

def strip_row(text):
    text = text.strip()
    text = ' '.join(text.split())
    return text

def first_name(list_text):
    name = ''
    if len(list_text) > 0:
      name = strip_row(list_text[0])
    return name

def check_first_author(row):
    if pd.isna(row['first_name']):
      return 'no'
    elif pd.isna(row['new_name']):
      return 'no'
    elif row['first_name'] == '':
      return 'no'
    elif row['first_name'] == row['new_name']:
      return 'yes'
    else:
      return 'no'

In [ ]:
#ทำการเปลี่ยนชื่อ คอลัมน์ ตามที่เราต้องการ
df_unique_name = df_unique_name.rename(columns={"clean_name": "authFullName"})

In [ ]:
#ทำการตรวจสอบชื่อจากคอลัมน์ที่ชื่อว่า authFullName ว่าเป็นภาษาอะไร แล้วเอาค่าที่ได้มาใส่ในคอลัมน์ที่ขื่อว่า language ซึ่งสามารถมีค่าได้ดังนี้
# th    คือภาษาไทย
# eng   คือภาษาอังกฤษ
# other คือที่นอกเหนือภาษาไทย และ ภาษาอังกฤษ
df_unique_name['language'] = df_unique_name['authFullName'].apply(language)

In [ ]:
# ทำการสร้าง dataframe เพื่อเก็บข้อมูลของชื่อในแต่ละภาษา
result_author_thai = df_unique_name[df_unique_name['language']=='th']

result_author_eng = df_unique_name[df_unique_name['language']=='eng']

result_author_other = df_unique_name[df_unique_name['language']=='other']

In [ ]:
result_author_thai

,authFullName,id,language
0,ไพฑูรย์ พงศะบุตร (ราชบัณฑิต),542598,th
1,ไพรัช ธัชยพงษ์ (ภาคีสมาชิก),542599,th
2,วิสุทธิ์ ใบไม้ (ราชบัณฑิต),542600,th
3,ระวี ภาวิไล (ราชบัณฑิต),542601,th
4,สนั่น ปัทมะทิน,542602,th
...,...,...,...
214,สิทธิ์ บุตรอินทร์ (ภาคีสมาชิก),542812,th
215,วิรุณ ตั้งเจริญ (ภาคีสมาชิก),542813,th
216,วรรณไวทยากร วรวรรณ,542814,th
217,สิริวัฒน์ วงษ์ศิริ (ภาคีสมาชิก),542815,th


In [ ]:
result_author_eng

,authFullName,id,language


In [ ]:
result_author_other

,authFullName,id,language


# Create Auth Detail

In [ ]:
# ทำการ join ตาราง df_unique_name กับ df เข้าด้วยกันเพื่อเอาค่าจากคอลัมน์ที่ชื่อว่า id ใน df_unique_name มาใส่ใน df

result_all = pd.merge(df_new,
                       df_unique_name,
                       left_on='clean_name',
                       right_on='authFullName',
                       how='left')

In [ ]:
result_all.columns

Index(['_index', '_type', '_id', '_score', '_source.subject', '_source.name',
       '_source.title', '_source.department', 'clean_name', 'prefix',
       'authFullName', 'id', 'language'],
      dtype='object')

In [ ]:
auth_detail = result_all

In [ ]:
#ทำการเปลี่ยนชื่อ คอลัมน์ ตามที่เราต้องการ
auth_detail = auth_detail.rename(columns={"id": "auth_ID","language":"author_lang","_source.title":"titleName", "_source.subject":"subject","_source.department":"department"})

In [ ]:
auth_detail.columns

Index(['_index', '_type', '_id', '_score', 'subject', '_source.name',
       'titleName', 'department', 'clean_name', 'prefix', 'authFullName',
       'auth_ID', 'author_lang'],
      dtype='object')

In [ ]:
# ทำการ Drop คอลัมน์ที่ไม่ใช้ออก
auth_detail = auth_detail.drop(['_index', '_type','_id', '_score',
       '_source.name','authFullName','clean_name','prefix'],axis=1)

In [ ]:
# ทำการสร้าง คอลัมน์ที่ยังไม่เคยมีขึ้นมา แล้วใส่ค่า Default
## auth_detail['id'] = ''
# auth_detail['auth_ID'] = ''
# auth_detail['author_lang'] = ''
# auth_detail['titleName'] = ''
auth_detail['tel'] = ''
auth_detail['email'] = ''
# auth_detail['subject'] = ''
# auth_detail['department'] = ''

In [ ]:
#สร้างตัวแปรที่ชื่อว่า all_column เพื่อไว้สำหรับจัดลำดับของ คอลัมน์ที่ต้องการ
all_column =  ["auth_ID","author_lang","titleName","tel","email","subject","department"]
len(all_column)

7

In [ ]:
len(auth_detail.columns)

7

In [ ]:
auth_detail.columns

Index(['subject', 'titleName', 'department', 'auth_ID', 'author_lang', 'tel',
       'email'],
      dtype='object')

In [ ]:
# จัดลำดับของคอลัมน์ใหม่ตามตัวแปรที่ชื่อว่า all_column
auth_detail = auth_detail[all_column]

In [ ]:
auth_detail

,auth_ID,author_lang,titleName,tel,email,subject,department
0,542598,th,ศาสตราจารย์กิตติคุณ,,,สังคมศาสตร์,ธรรมศาสตร์และการเมือง
1,542599,th,ศาสตราจารย์ ดร.,,,วิทยาศาสตร์ประยุกต์,วิทยาศาสตร์
2,542600,th,ศาสตราจารย์เกียรติคุณ ดร.,,,วิทยาศาสตร์ชีวภาพ,วิทยาศาสตร์
3,542601,th,ศาสตราจารย์กิตติคุณ ดร.,,,วิทยาศาสตร์กายภาพ,วิทยาศาสตร์
4,542602,th,ศาสตราจารย์,,,สังคมศาสตร์,ธรรมศาสตร์และการเมือง
...,...,...,...,...,...,...,...
214,542812,th,ศาสตราจารย์ ดร.,,,ปรัชญา,ธรรมศาสตร์และการเมือง
215,542813,th,ศาสตราจารย์ ดร.,,,วิจิตรศิลป์,ศิลปกรรม
216,542814,th,พลตรี พระเจ้าวรวงศ์เธอ กรมหมื่นนราธิปพงศ์ประพั...,,,ปรัชญา,ธรรมศาสตร์และการเมือง
217,542815,th,ศาสตราจารย์ ดร.,,,วิทยาศาสตร์ชีวภาพ,วิทยาศาสตร์


# Author Expert

In [ ]:
auth_expert = result_all

In [ ]:
auth_expert.columns

Index(['_index', '_type', '_id', '_score', '_source.subject', '_source.name',
       '_source.title', '_source.department', 'clean_name', 'prefix',
       'authFullName', 'id', 'language'],
      dtype='object')

In [ ]:
#ทำการเปลี่ยนชื่อ คอลัมน์ ตามที่เราต้องการ
auth_expert = auth_expert.rename(columns={"id": "auth_ID","language":"author_lang", "_source.subject":"subject" })

In [ ]:
auth_expert.columns

Index(['_index', '_type', '_id', '_score', 'subject', '_source.name',
       '_source.title', '_source.department', 'clean_name', 'prefix',
       'authFullName', 'auth_ID', 'author_lang'],
      dtype='object')

In [ ]:
# ทำการ Drop คอลัมน์ที่ไม่ใช้ออก
auth_expert = auth_expert.drop(['_index', '_type','_id', '_score',
       '_source.name','_source.title','_source.department','authFullName','clean_name','prefix'],axis=1)

In [ ]:
# ทำการสร้าง คอลัมน์ที่ยังไม่เคยมีขึ้นมา แล้วใส่ค่า Default
## auth_expert['id'] = ''
# auth_expert['auth_ID'] = ''
# auth_expert['author_lang'] = ''
# auth_expert['subject'] = ''
auth_expert['expertise'] = ''

In [ ]:
#สร้างตัวแปรที่ชื่อว่า all_column เพื่อไว้สำหรับจัดลำดับของ คอลัมน์ที่ต้องการ
all_column =  ["auth_ID","author_lang","subject","expertise"]
len(all_column)

4

In [ ]:
len(auth_expert.columns)

4

In [ ]:
auth_expert.columns

Index(['subject', 'auth_ID', 'author_lang', 'expertise'], dtype='object')

In [ ]:
# จัดลำดับของคอลัมน์ใหม่ตามตัวแปรที่ชื่อว่า all_column
auth_expert = auth_expert[all_column]

In [ ]:
auth_expert

,auth_ID,author_lang,subject,expertise
0,542598,th,สังคมศาสตร์,
1,542599,th,วิทยาศาสตร์ประยุกต์,
2,542600,th,วิทยาศาสตร์ชีวภาพ,
3,542601,th,วิทยาศาสตร์กายภาพ,
4,542602,th,สังคมศาสตร์,
...,...,...,...,...
214,542812,th,ปรัชญา,
215,542813,th,วิจิตรศิลป์,
216,542814,th,ปรัชญา,
217,542815,th,วิทยาศาสตร์ชีวภาพ,


# Export

In [ ]:
# author

result_author_thai[['id','authFullName']].to_csv('royin_author_thai.csv')
result_author_eng[['id','authFullName']].to_csv('royin_author_eng.csv')
result_author_other[['id','authFullName']].to_csv('royin_author_other.csv')

In [ ]:
# author detail , education, expert
auth_detail.index.name = 'id'
auth_expert.index.name = 'id'

auth_detail.to_csv('royin_author_detail.csv')
auth_expert.to_csv('royin_author_expert.csv')